In [ ]:
import numpy as np
import pandas as pd
import warnings
import json
import joblib
import time
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import optuna
import torch

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)
np.random.seed(42)

# =============================================================================
# GPU CONFIGURATION
# =============================================================================
print("="*80)
print("LIGHTGBM HYPERPARAMETER OPTIMIZATION")
print("="*80)
if torch.cuda.is_available():
    print(f"✓ GPU detected: {torch.cuda.get_device_name(0)}")
    USE_GPU = True
else:
    print("⚠ No GPU detected, using CPU")
    USE_GPU = False

# =============================================================================
# LOAD PREPROCESSED DATA
# =============================================================================
print("\n" + "="*80)
print("LOADING PREPROCESSED DATA")
print("="*80)

X_train = joblib.load('../../data/preprocessed/X_train.pkl')
y_train = joblib.load('../../data/preprocessed/y_train.pkl')
metadata = joblib.load('../../data/preprocessed/metadata.pkl')

print(f"✓ Train shape: {X_train.shape}")
print(f"✓ Target shape: {y_train.shape}")
print(f"✓ GPU available: {USE_GPU}")

# =============================================================================
# OPTIMIZATION SETTINGS
# =============================================================================
FAST_MODE = True  # Set to False for more thorough search
N_TRIALS = 20 if FAST_MODE else 50
N_FOLDS = 5

print(f"\n{'='*60}")
print(f"OPTIMIZATION SETTINGS")
print(f"{'='*60}")
print(f"Mode: {'FAST' if FAST_MODE else 'THOROUGH'}")
print(f"Trials: {N_TRIALS}")
print(f"Folds: {N_FOLDS}")

# =============================================================================
# OBJECTIVE FUNCTION
# =============================================================================
def objective_lgb(trial, X, y):
    """Objective function for LightGBM optimization"""
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'device': 'gpu' if USE_GPU else 'cpu',
        'gpu_use_dp': False if USE_GPU else None,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 60),
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.95),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.95),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 1.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 1.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 1.0),
        'seed': 42
    }
    params = {k: v for k, v in params.items() if v is not None}

    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        train_data = lgb.Dataset(X_tr, label=y_tr)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        model = lgb.train(
            params, train_data, num_boost_round=1000,
            valid_sets=[val_data],
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
        )

        preds = np.clip(model.predict(X_val), 0, 100)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        scores.append(rmse)

    return np.mean(scores)

# =============================================================================
# RUN OPTIMIZATION
# =============================================================================
print("\n" + "="*80)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*80)

start_time = time.time()

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

study.optimize(
    lambda trial: objective_lgb(trial, X_train, y_train),
    n_trials=N_TRIALS,
    show_progress_bar=True,
    n_jobs=1
)

optimization_time = time.time() - start_time

# =============================================================================
# RESULTS
# =============================================================================
print("\n" + "="*80)
print("OPTIMIZATION RESULTS")
print("="*80)
print(f"Best RMSE: {study.best_value:.6f}")
print(f"Optimization time: {optimization_time:.1f}s")
print(f"\nBest parameters:")
for param, value in study.best_params.items():
    print(f"  {param:20s}: {value}")

# =============================================================================
# SAVE RESULTS
# =============================================================================
import os
os.makedirs('../../models', exist_ok=True)

# Save best parameters
best_params = study.best_params.copy()
best_params.update({
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'device': 'gpu' if USE_GPU else 'cpu',
    'gpu_use_dp': False if USE_GPU else None,
    'seed': 42
})
best_params = {k: v for k, v in best_params.items() if v is not None}

joblib.dump(best_params, '../../models/lightgbm_params.pkl')
print("\n✓ Parameters saved to: ../../models/lightgbm_params.pkl")

# Save optimization history
history = pd.DataFrame({
    'trial': [t.number for t in study.trials],
    'value': [t.value for t in study.trials],
    'params': [str(t.params) for t in study.trials]
})
history.to_csv('../../models/lightgbm_history.csv', index=False)
print("✓ History saved to: ../../models/lightgbm_history.csv")

# Save study object
joblib.dump(study, '../../models/lightgbm_study.pkl')
print("✓ Study saved to: ../../models/lightgbm_study.pkl")

# Save summary
summary = {
    'model': 'LightGBM',
    'best_rmse': study.best_value,
    'n_trials': N_TRIALS,
    'n_folds': N_FOLDS,
    'optimization_time': optimization_time,
    'use_gpu': USE_GPU,
    'best_params': best_params
}
joblib.dump(summary, '../../models/lightgbm_summary.pkl')
print("✓ Summary saved to: ../../models/lightgbm_summary.pkl")

print("\n" + "="*80)
print("LIGHTGBM OPTIMIZATION COMPLETE!")
print("="*80)
print(f"✓ Best CV RMSE: {study.best_value:.6f}")
print(f"✓ All results saved in '../../models/'")

LIGHTGBM HYPERPARAMETER OPTIMIZATION
⚠ No GPU detected, using CPU

LOADING PREPROCESSED DATA
✓ Train shape: (630000, 9)
✓ Target shape: (630000,)
✓ GPU available: False

OPTIMIZATION SETTINGS
Mode: FAST
Trials: 20
Folds: 4

STARTING BAYESIAN OPTIMIZATION


  0%|          | 0/20 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[993]	valid_0's rmse: 8.82734
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[999]	valid_0's rmse: 8.83552
[W 2026-01-10 15:47:42,096] Trial 0 failed with parameters: {'learning_rate': 0.04370861069626263} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/lorenzo/Documenti/datascience_projects/students-scores/.venv/lib/python3.13/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_10565/1394054503.py", line 122, in <lambda>
    lambda trial: objective_lgb(trial, X_train, y_train),
                  ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_10565/1394054503.py", line 101, in objective_lgb
    preds = np.clip(model.predict(X_val), 0, 100)
                    ~~~~~~~~~~~~~^^^^^^^


KeyboardInterrupt: 